# Walkthrough: DataFrames in Spark



# 1. Overview

What is Spark SQL?
- Spark SQL takes basic RDDs and **puts a schema on them**.

What is a DataFrame?
- DataFrames are the primary abstraction in Spark SQL.
- Think of a DataFrames as **RDDs with schema**.

What are **schemas**?
- Schemas are metadata about your data.
- Schema = Table Names + Column Names + Column Types

What are the pros of schemas?
- Schemas enable using **column names** instead of column positions
- Schemas enable **queries** using SQL and DataFrame syntax
- Schemas make your data more **structured**.

# 2. Operational DataFrames in Python

We'll proceed along the usual spark flow (see above).
1. create the enviromnent to run Spark SQL from python
2. create DataFrames from RDDs or from files
3. run some transformations
4. execute actions to obtain values (local objects in python)

## 2.1. Initializing a `SparkSession` (`SparkContext` and `SqlContext`) in Python

Using:

```python
import pyspark as ps

spark = ps.sql.SparkSession.builder \
            .master("local[4]") \
            .appName("df lecture") \
            .getOrCreate()
```

will create a *"local"* cluster made of the driver using all 4 cores.

In [1]:
import pyspark as ps

spark = ps.sql.SparkSession.builder \
            .master("local[4]") \
            .appName("df lecture") \
            .getOrCreate()

sc = spark.sparkContext

In [2]:
spark

## 2.2. Creating a DataFrame

### 2.2.1. From an RDD (specifying schema)

You can create a DataFrame from an existing RDD (whatever source you used to create this one), if you add a schema.

To build a schema, you will use existing data types provided in the [`pyspqrk.sql.types`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.types) module. Here's a list of the most useful ones (subjective criteria).

| Types | Python-like type |
| - | - |
| StringType | string |
| IntegerType | int |
| FloatType | float |
| ArrayType\* | array or list |
| MapType | dict |

\* see later UDF functions on how to use that

In [3]:
%ls data

aapl.csv                  toy_dataB.txt
airline-data-extract.csv  toy_data.pkl/
cookie_data.txt           toy_data.txt
input.txt                 transactions.txt
sales2.json.gz            users.txt
sales.csv                 yelp_academic_dataset_business_extract.json
sales.json                yelp_academic_dataset_business.json.gz
sales.txt


In [4]:
# remember that csv file ?
def casting_function((id, date, store, state, product, amount)):
    return((int(id), date, int(store), state, int(product), float(amount)))

rdd_sales = sc.textFile('data/sales.txt')\
        .map(lambda rowstr : rowstr.split())\
        .filter(lambda row: not row[0].startswith('#'))\
        .map(casting_function)

rdd_sales.collect()

[(101, u'11/13/2014', 100, u'WA', 331, 300.0),
 (104, u'11/18/2014', 700, u'OR', 329, 450.0),
 (102, u'11/15/2014', 203, u'CA', 321, 200.0),
 (106, u'11/19/2014', 202, u'CA', 331, 330.0),
 (103, u'11/17/2014', 101, u'WA', 373, 750.0),
 (105, u'11/19/2014', 202, u'CA', 321, 200.0)]

In [5]:
# import the many data types
from pyspark.sql.types import *

# create a schema of your own
schema = StructType( [
    StructField('id',IntegerType(),True),
    StructField('date',StringType(),True),
    StructField('store',IntegerType(),True),
    StructField('state',StringType(),True),
    StructField('product',IntegerType(),True),
    StructField('amount',FloatType(),True) ] )

# feed that into a DataFrame
df = spark.createDataFrame(rdd_sales,schema)

# show the result
df.show()

# print the schema
df.printSchema()

+---+----------+-----+-----+-------+------+
| id|      date|store|state|product|amount|
+---+----------+-----+-----+-------+------+
|101|11/13/2014|  100|   WA|    331| 300.0|
|104|11/18/2014|  700|   OR|    329| 450.0|
|102|11/15/2014|  203|   CA|    321| 200.0|
|106|11/19/2014|  202|   CA|    331| 330.0|
|103|11/17/2014|  101|   WA|    373| 750.0|
|105|11/19/2014|  202|   CA|    321| 200.0|
+---+----------+-----+-----+-------+------+

root
 |-- id: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- store: integer (nullable = true)
 |-- state: string (nullable = true)
 |-- product: integer (nullable = true)
 |-- amount: float (nullable = true)



### 2.2.2. Reading from files (infering schema)

Use [`sqlContext.read.csv`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.csv) to load a CSV into a DataFrame. You can specify every useful parameter in there. It can infer the schema.

In [6]:
# read CSV
df = spark.read.csv('data/aapl.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

# prints the schema
df.printSchema()

# some functions are still valid
print("line count: {}".format(df.count()))

# show the table in a oh-so-nice format
df.show()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

line count: 253
+--------------------+----------+----------+----------+----------+--------+----------+
|                Date|      Open|      High|       Low|     Close|  Volume| Adj Close|
+--------------------+----------+----------+----------+----------+--------+----------+
|2016-10-25 00:00:...|117.949997|118.360001|117.309998|    118.25|39190300|    118.25|
|2016-10-24 00:00:...|117.099998|117.739998|     117.0|117.650002|23538700|117.650002|
|2016-10-21 00:00:...|116.809998|116.910004|116.279999|116.599998|23192700|116.599998|
|2016-10-20 00:00:...|116.860001|117.379997|116.330002|117.059998|24125800|117.059998|
|2016-10-19 00:00:...|    117.25|117.760002|113.800003|117.120003|20034600|117.120003|
|2016-10-18 00:00:.

Use [`sqlContext.read.json`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json) to load a JSON file into a DataFrame. You can specify every useful parameter in there. It can infer the schema.

In [7]:
%ls data

aapl.csv                  toy_dataB.txt
airline-data-extract.csv  toy_data.pkl/
cookie_data.txt           toy_data.txt
input.txt                 transactions.txt
sales2.json.gz            users.txt
sales.csv                 yelp_academic_dataset_business_extract.json
sales.json                yelp_academic_dataset_business.json.gz
sales.txt


In [8]:
# read JSON
df = spark.read.json('data/sales.json')

# prints the schema
df.printSchema()

# some functions are still valid
print("line count: {}".format(df.count()))

# show the table in a oh-so-nice format
df.show()

root
 |-- amount: double (nullable = true)
 |-- date: string (nullable = true)
 |-- id: long (nullable = true)
 |-- product: long (nullable = true)
 |-- state: string (nullable = true)
 |-- store: long (nullable = true)

line count: 6
+------+----------+---+-------+-----+-----+
|amount|      date| id|product|state|store|
+------+----------+---+-------+-----+-----+
| 300.0|11/13/2014|101|    331|   WA|  100|
| 450.0|11/18/2014|104|    329|   OR|  700|
| 200.0|11/15/2014|102|    321|   CA|  203|
| 330.0|11/19/2014|106|    331|   CA|  202|
| 750.0|11/17/2014|103|    373|   WA|  101|
| 200.0|11/19/2014|105|    321|   CA|  202|
+------+----------+---+-------+-----+-----+



In [9]:
# read JSON
df = spark.read.json('data/sales2.json.gz')

# show the table in a oh-so-nice format
df.show()

+------+----------+---+-------+-----+-----+
|amount|      date| id|product|state|store|
+------+----------+---+-------+-----+-----+
| 300.0|11/13/2014|101|    331|   WA|  100|
| 450.0|11/18/2014|104|    329|   OR|  700|
| 200.0|11/15/2014|102|    321|   CA|  203|
| 330.0|11/19/2014|106|    331|   CA|  202|
| 750.0|11/17/2014|103|    373|   WA|  101|
| 200.0|11/19/2014|105|    321|   CA|  202|
+------+----------+---+-------+-----+-----+



## 2.3. Actions : turning your DataFrame into a local object

Some actions just remain the same, you won't have to learn Spark all over again.

Some new actions give you the possibility to describe and show the content in a more fashionable manner.

When used/executed in IPython or in a notebook, they **launch the processing of the DAG**. This is where Spark stops being **lazy**. This is where your script will take time to execute.

| Method | DF vs RDD? | Description |
| - | - | - |
| [`.collect()`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.collect) | identical | Return a list that contains all of the elements as Rows. |
| [`.count()`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.count) | identical | Return the number of elements. |
| [`.take(n)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.take) | identical | Take the first `n` elements. |
| [`.top(n)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.top) | identical | Get the top `n` elements. |
| [`.first()`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.first) | identical | Return the first element. |
| [`.show(n)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.show) | <span style="color:green">new</span> | Show the DataFrame in table format (`n=20` by default) |
| [`.toPandas()`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.toPandas) | <span style="color:green">new</span> | Convert the DF into a Pandas DF. |
| [`.printSchema(*cols)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.printSchema)\* | <span style="color:green">new</span> | Display the schema. This is not an action, it doesn't launch the DAG, but it fits better in this category. |
| [`.describe(*cols)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.describe) | <span style="color:green">new</span> | Compute statistics for this column. |
| [`.sum(*cols)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.sum) | <span style="color:red">different</span> | Applies on GroupedData only (see transformations). |
| [`.mean(*cols)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.mean) | <span style="color:red">different</span> | Applies on GroupedData only (see transformations). |
| [`.min(*cols)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.min) | <span style="color:red">different</span> | Applies on GroupedData only (see transformations). |
| [`.max(*cols)`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.max) | <span style="color:red">different</span> | Applies on GroupedData only (see transformations). |


In [10]:
# read CSV
df_sales = spark.read.csv('data/sales.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

In [11]:
df_sales.show()

+---+----------+-----+-----+-------+------+
|#ID|      Date|Store|State|Product|Amount|
+---+----------+-----+-----+-------+------+
|101|11/13/2014|  100|   WA|    331| 300.0|
|104|11/18/2014|  700|   OR|    329| 450.0|
|102|11/15/2014|  203|   CA|    321| 200.0|
|106|11/19/2014|  202|   CA|    331| 330.0|
|103|11/17/2014|  101|   WA|    373| 750.0|
|105|11/19/2014|  202|   CA|    321| 200.0|
+---+----------+-----+-----+-------+------+



In [12]:
df_sales.toPandas()

,#ID,Date,Store,State,Product,Amount
0,101,11/13/2014,100,WA,331,300.0
1,104,11/18/2014,700,OR,329,450.0
2,102,11/15/2014,203,CA,321,200.0
3,106,11/19/2014,202,CA,331,330.0
4,103,11/17/2014,101,WA,373,750.0
5,105,11/19/2014,202,CA,321,200.0


This is how `.collect()` returns things...

In [13]:
df_sales.collect()[0]["Date"]

u'11/13/2014'

In [14]:
# prints the schema
print("--- printSchema()")
df_sales.printSchema()

# prints the table itself
print("--- show()")
df_sales.show()

# show the statistics of all numerical columns
print("--- describe()")
df_sales.describe().show()

# show the statistics of one specific column
print("--- describe(Amount)")
df_sales.describe("Amount").show()

--- printSchema()
root
 |-- #ID: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- Store: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- Product: integer (nullable = true)
 |-- Amount: double (nullable = true)

--- show()
+---+----------+-----+-----+-------+------+
|#ID|      Date|Store|State|Product|Amount|
+---+----------+-----+-----+-------+------+
|101|11/13/2014|  100|   WA|    331| 300.0|
|104|11/18/2014|  700|   OR|    329| 450.0|
|102|11/15/2014|  203|   CA|    321| 200.0|
|106|11/19/2014|  202|   CA|    331| 330.0|
|103|11/17/2014|  101|   WA|    373| 750.0|
|105|11/19/2014|  202|   CA|    321| 200.0|
+---+----------+-----+-----+-------+------+

--- describe()
+-------+------------------+----------+------------------+-----+------------------+------------------+
|summary|               #ID|      Date|             Store|State|           Product|            Amount|
+-------+------------------+----------+------------------+-----+--------------

## 2.3. Transformations on DataFrames

- They are still **lazy**: Spark doesn't apply the transformation right away, it just builds on the **DAG**
- They transform a DataFrame into another because DataFrames are also **immutable**.
- They can be **wide** or **narrow** (whether they shuffle partitions or not).

You got that... DataFrames are just RDDs with a schema.

| Method | Type | Category | Description |
| - | - | - |
| [`.withColumn(label,func)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.withColumn) | transformation | mapping | Returns a new DataFrame by adding a column or replacing the existing column that has the same name. |
| [`.filter(condition)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.filter) | transformation | reduction |  Filters rows using the given condition. |
| [`.sample()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.sample) | transformation | reduction | Return a sampled subset of this DataFrame. |
| [`.sampleBy(col)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.sampleBy) | transformation | reduction | Returns a stratified sample without replacement based on the fraction given on each stratum. |
| [`.select(cols)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.select) | transformation | reduction | Projects a set of expressions and returns a new DataFrame. |
| [`.join(dfB)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.join) | transformation | operations | Joins with another DataFrame, using the given join expression. |
| [`.groupBy(col)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy) | transformation | operations | Groups the DataFrame using the specified columns, so we can run aggregation on them.  |
| [`.sort(cols)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.sort) | transformation | sorting |  Returns a new DataFrame sorted by the specified column(s). |



#### `.withColumn("label", func)` : 

In [15]:
# read CSV
df_aapl = spark.read.csv('data/aapl.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

df_aapl.show(5)

df_aapl.printSchema()

+--------------------+----------+----------+----------+----------+--------+----------+
|                Date|      Open|      High|       Low|     Close|  Volume| Adj Close|
+--------------------+----------+----------+----------+----------+--------+----------+
|2016-10-25 00:00:...|117.949997|118.360001|117.309998|    118.25|39190300|    118.25|
|2016-10-24 00:00:...|117.099998|117.739998|     117.0|117.650002|23538700|117.650002|
|2016-10-21 00:00:...|116.809998|116.910004|116.279999|116.599998|23192700|116.599998|
|2016-10-20 00:00:...|116.860001|117.379997|116.330002|117.059998|24125800|117.059998|
|2016-10-19 00:00:...|    117.25|117.760002|113.800003|117.120003|20034600|117.120003|
+--------------------+----------+----------+----------+----------+--------+----------+
only showing top 5 rows

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |--

#### `.withColumn("label", func)` : constant value

In [16]:
from pyspark.sql.functions import lit

df_out = df_aapl.withColumn("blabla", lit("echo"))

df_out.show(5)

+--------------------+----------+----------+----------+----------+--------+----------+------+
|                Date|      Open|      High|       Low|     Close|  Volume| Adj Close|blabla|
+--------------------+----------+----------+----------+----------+--------+----------+------+
|2016-10-25 00:00:...|117.949997|118.360001|117.309998|    118.25|39190300|    118.25|  echo|
|2016-10-24 00:00:...|117.099998|117.739998|     117.0|117.650002|23538700|117.650002|  echo|
|2016-10-21 00:00:...|116.809998|116.910004|116.279999|116.599998|23192700|116.599998|  echo|
|2016-10-20 00:00:...|116.860001|117.379997|116.330002|117.059998|24125800|117.059998|  echo|
|2016-10-19 00:00:...|    117.25|117.760002|113.800003|117.120003|20034600|117.120003|  echo|
+--------------------+----------+----------+----------+----------+--------+----------+------+
only showing top 5 rows



#### `.withColumn("label", func)` : column operations

In [17]:
from pyspark.sql.functions import col

#df_out = df_aapl.withColumn("diff", df_aapl.High - df_aapl.Low)
df_out = df_aapl.withColumn("diff", df_aapl["High"] - df_aapl["Low"])
df_out.show(5)

df_out = df_aapl.withColumn("diff", df_aapl.High - df_aapl.Low)
df_out.show(5)

# below is the PREFERED METHOD for referencing columns
df_out = df_aapl.withColumn("diff", col("High") - col("Low"))
df_out.show(5)

+--------------------+----------+----------+----------+----------+--------+----------+------------------+
|                Date|      Open|      High|       Low|     Close|  Volume| Adj Close|              diff|
+--------------------+----------+----------+----------+----------+--------+----------+------------------+
|2016-10-25 00:00:...|117.949997|118.360001|117.309998|    118.25|39190300|    118.25|1.0500030000000038|
|2016-10-24 00:00:...|117.099998|117.739998|     117.0|117.650002|23538700|117.650002|0.7399979999999999|
|2016-10-21 00:00:...|116.809998|116.910004|116.279999|116.599998|23192700|116.599998| 0.630004999999997|
|2016-10-20 00:00:...|116.860001|117.379997|116.330002|117.059998|24125800|117.059998|1.0499950000000098|
|2016-10-19 00:00:...|    117.25|117.760002|113.800003|117.120003|20034600|117.120003|3.9599989999999963|
+--------------------+----------+----------+----------+----------+--------+----------+------------------+
only showing top 5 rows

+--------------------

#### `.withColumn("label", func)` : user defined function

In [18]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import DoubleType

def my_specialfunc(h,l,o,c):
    return ((h-l)*(o-c))

my_specialfunc_udf = udf(lambda h,l,o,c : my_specialfunc(h,l,o,c), DoubleType())

df_out = df_aapl.withColumn("special",
                            my_specialfunc_udf(col('High'),
                                               col('Low'),
                                               col('Open'),
                                               col('Close')))

df_out.show()

+--------------------+----------+----------+----------+----------+--------+----------+--------------------+
|                Date|      Open|      High|       Low|     Close|  Volume| Adj Close|             special|
+--------------------+----------+----------+----------+----------+--------+----------+--------------------+
|2016-10-25 00:00:...|117.949997|118.360001|117.309998|    118.25|39190300|    118.25| -0.3150040500090051|
|2016-10-24 00:00:...|117.099998|117.739998|     117.0|117.650002|23538700|117.650002| -0.4070018599920009|
|2016-10-21 00:00:...|116.809998|116.910004|116.279999|116.599998|23192700|116.599998| 0.13230104999999545|
|2016-10-20 00:00:...|116.860001|117.379997|116.330002|117.059998|24125800|117.059998|-0.20999585001499796|
|2016-10-19 00:00:...|    117.25|117.760002|113.800003|117.120003|20034600|117.120003|  0.5147879900030115|
|2016-10-18 00:00:...|    118.18|118.209999|117.449997|117.470001|24553500|117.470001|   0.539600659998008|
|2016-10-17 00:00:...|117.33

#### `.select(*cols)` : selecting specific columns

In [19]:
df_out = df_aapl.select(["Open", "Close"])

df_out.show(5)

+----------+----------+
|      Open|     Close|
+----------+----------+
|117.949997|    118.25|
|117.099998|117.650002|
|116.809998|116.599998|
|116.860001|117.059998|
|    117.25|117.120003|
+----------+----------+
only showing top 5 rows



#### `.groupBy()`: aggregating in DataFrames

In [20]:
from pyspark.sql import functions as F

df_out = df_sales.groupBy(col("State")).agg(F.sum(col("Amount")),F.mean("Amount"))

df_out.show()

+-----+-----------+------------------+
|State|sum(Amount)|       avg(Amount)|
+-----+-----------+------------------+
|   OR|      450.0|             450.0|
|   CA|      730.0|243.33333333333334|
|   WA|     1050.0|             525.0|
+-----+-----------+------------------+



#### `.orderBy()` : sorting by a column

In [21]:
df_out = df_sales.groupBy(col("State"))\
            .agg(F.sum(col("Amount")).alias("sumAmount"))\
            .orderBy(col("sumAmount"), ascending=False)

df_out.show()

+-----+---------+
|State|sumAmount|
+-----+---------+
|   WA|   1050.0|
|   CA|    730.0|
|   OR|    450.0|
+-----+---------+



## 2.4. Execute SQL statements

#### .createOrReplaceTempView(name) : registering your dataframe as a table

In [22]:
# read CSV
df_sales = spark.read.csv('data/sales.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

# Now create an SQL table and issue SQL queries against it without
# using the sqlContext but through the SparkSession object.
# Creates a temporary view of the DataFrame
df_sales.createOrReplaceTempView("sales")

In [23]:
result = spark.sql('''
    SELECT state, AVG(amount) as avg_amount
    FROM sales
    GROUP BY state
    ''')
result.show()

+-----+------------------+
|state|        avg_amount|
+-----+------------------+
|   OR|             450.0|
|   CA|243.33333333333334|
|   WA|             525.0|
+-----+------------------+



### spark.udf.register(name, func) : register a user defined function

In [24]:
def myfun(x):
    return x**2

spark.udf.register('my_sql_fun', myfun)

result = spark.sql('''
    SELECT state, my_sql_fun(amount) as square_amount
    FROM sales
    ''')

result.show()

+-----+-------------+
|state|square_amount|
+-----+-------------+
|   WA|      90000.0|
|   OR|     202500.0|
|   CA|      40000.0|
|   CA|     108900.0|
|   WA|     562500.0|
|   CA|      40000.0|
+-----+-------------+



# 3. Let's design chains of transformations together ! (reloaded)

## 3.1. Computing sales per state

### Input DataFrame

In [25]:
# read CSV
df_sales = spark.read.csv('data/sales.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

df_sales.show()

+---+----------+-----+-----+-------+------+
|#ID|      Date|Store|State|Product|Amount|
+---+----------+-----+-----+-------+------+
|101|11/13/2014|  100|   WA|    331| 300.0|
|104|11/18/2014|  700|   OR|    329| 450.0|
|102|11/15/2014|  203|   CA|    321| 200.0|
|106|11/19/2014|  202|   CA|    331| 330.0|
|103|11/17/2014|  101|   WA|    373| 750.0|
|105|11/19/2014|  202|   CA|    321| 200.0|
+---+----------+-----+-----+-------+------+



### Task

You want to obtain an ~~RDD~~ DataFrame of the states sorted by their decreasing cumulated sales.

What transformations do you need to apply ?

If you had to draw a workflow of the transformations to apply ?

### Code

In [43]:
df_out = df_sales # code your transformations here
df_out = df_out.groupBy(col('State')).agg(F.sum(col('Amount'))).alias('Money')
df_out.show()

+-----+-----------+
|State|sum(Amount)|
+-----+-----------+
|   OR|      450.0|
|   CA|      730.0|
|   WA|     1050.0|
+-----+-----------+



### Solution

<details>
  <summary>Click here to see the solution below</summary>
```
df_out = df_sales.groupBy(col('State'))\
                 .agg(F.sum(col('Amount')).alias('Money'))\
                 .orderBy("Money", ascending=False)

df_out.show()
```
</details>

## 3.2. Find the date on which AAPL's stock price was the highest

### Input DataFrame

In [ ]:
# read CSV
df_aapl = spark.read.csv('data/aapl.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

df_aapl.show(5)

### Task

Now, design a pipeline that would :

1. ~~filter out headers and last line~~
2. ~~split each line based on comma~~
3. keep only fields for Date ~~(col 0)~~ and Close ~~(col 4)~~
4. order by Close in descending order

### Code

In [ ]:
df_out = df_aapl  # put your transformation here...

df_out.show(5)

### Solution

<details>
  <summary>Click here to see the solution below</summary>
```
df_out.select("Close", "Date").orderBy(df_aapl.Close, ascending=False).show(5)```
</details>